# SCC-DFTB Dataset generation
Generate DFTB_DB from existing database of DFT structures for fitting CCS. In this book, the calcualtions are done in serial. For 

In [1]:
from ase.io import read,write
from ase.io.vasp import read_vasp_out
from ase.atoms import *
from ase.calculators.dftb import Dftb
from ase.calculators.vasp import Vasp
from ase.optimize import *
import ase.db
import sys, os
import numpy as np
from ase.db import connect
os.environ['DFTB_COMMAND'] = '/home/broqvist/RUN_DFTB+/dftbplus-20.2/bin/dftb+'
base_dir=os.getcwd()
os.chdir(base_dir) 
if not os.path.isdir("RUN_SK/"):
    os.mkdir("RUN_SK/")

def setup_model(elements,spin,charge,**kwargs):
    print('setting up calculator')
    os.environ["DFTB_PREFIX"] = base_dir+"/SKF/"
    DFTB_calc=Dftb(Hamiltonian_SCC='Yes',Hamiltonian_ShellResolvedSCC = 'Yes',Hamiltonian_SCCTolerance = '1.0E-006',Hamiltonian_Mixer= 'Anderson{',Hamiltonian_Mixer_MixingParameter='0.03',Hamiltonian_MaxSCCIterations = '150',Hamiltonian_Filling = 'MethfesselPaxton{',Hamiltonian_Filling_Temperature = '0.001583407672623195',Hamiltonian_KPointsAndWeights = 'SupercellFolding{1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}',) 
    if spin>0:                  
        DFTB_calc.set(Hamiltonian_SpinPolarisation = 'Colinear{') 
        DFTB_calc.set(Hamiltonian_SpinPolarisation_UnpairedElectrons=str(spin))
        DFTB_calc.set(Hamiltonian_SpinConstants_ ='')
        for i in elements: 
            print(i)
            spinparam=get_spindata(i)
            string=f'Hamiltonian_SpinConstants_{i}'  
            print(string)
            DFTB_calc.set(**{string:spinparam})  
    if charge>0: 
        DFTB_calc.set(Hamiltonian_Charge =str(charge))
    calc=DFTB_calc
    return calc


def get_data(element): 
    if element=='Li':
        maxang="'p'"
    if element=='C':
        maxang="'p'"
    if element=='O':
        maxang="'p'"
    if element=='H':
        maxang="'s'"
    if element=='P':
        maxang="'p'"    
    return maxang                 

def get_spindata(element): 
    if element=='Li':
        spinparam='{-0.019  -0.016  -0.016 -0.027}'
    if element=='C':
        spinparam='{-0.031 -0.025 -0.025 -0.023}'
    if element=='O':
        spinparam='{-0.035 -0.030 -0.030 -0.028}'
    if element=='H':
        spinparam='{-0.072}'
    if element=='P':
        spinparam='{-0.035 -0.030 -0.030 -0.0280}'   
    return spinparam        
#os.chdir(base_dir)
print('You are here:', base_dir)


You are here: /home/broqvist/Desktop/parautomatik/EC


In [2]:

def run_dftb(structure,SKF):
    os.chdir(base_dir+'/RUN_SK')

    w = structure.get_chemical_symbols()
    print('w is',w)
    charge=0
    spin=0
    calculator = setup_model(w,spin,charge)
    structure.calc=calculator
    structure.get_potential_energy()
    structure.get_forces()
    os.chdir(base_dir)
    return structure    

def create_id_list(db_dft):
    idlist=[]
    for row in db_pbe.select(): 
        if row.DFTB==False:   
        
            idlist.append(row.key) 
        
    return idlist

In [ ]:
os.chdir(base_dir)

SKF=base_dir+"/SKF/"
dbname_DFT=base_dir+'/DFT_DB.db'
dbname_DFTB=base_dir+'/DFTB_DB.db'
db_dftb=ase.db.connect(dbname_DFTB)
db_dft=ase.db.connect(dbname_DFT)

#idlist=create_id_list()  
#loc=np.random.randint(1,21,21)
#print(loc)



for row in db_dft.select():
    structure=row.toatoms()
    print(structure)
    try: 
        structure=run_dftb(structure,SKF)
        db_dft.update(row.id, key=row.id, DFTB=True)
        print('DFT_DB updated')
        db_dftb.write(structure,key=row.id)
        print("structure", row.key, "written to DFTB_db \"")
    except:
        print('Error')

Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 1 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 2 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27064 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 4 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w 

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


DFT_DB updated
Error
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27237 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27240 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27254 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27235 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculat

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


DFT_DB updated
Error
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27388 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27300 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27354 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27355 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculat

...

DFT_DB updated
Error
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27109 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27122 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27143 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27144 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculat

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


DFT_DB updated
Error
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27338 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27339 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27340 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27323 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculat

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


DFT_DB updated
Error
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27467 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27468 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27482 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculator=SinglePointCalculator(...))
w is ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'O', 'O', 'O']
setting up calculator
DFT_DB updated
structure 27492 written to DFTB_db "
Atoms(symbols='C3H4O3', pbc=True, cell=[21.0, 20.0, 20.0], calculat

In [ ]:
!ase db DFT_DB.db
